In [1]:
from time import sleep
from random import randint

In [2]:
from time import time
start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.4987242668487976 requests/s
Request: 2; Frequency: 0.39930871476182256 requests/s
Request: 3; Frequency: 0.42798091660459886 requests/s
Request: 4; Frequency: 0.44370639788513166 requests/s
Request: 5; Frequency: 0.4990085155329949 requests/s


In [3]:
from IPython.core.display import clear_output
start_time = time()
requests = 0
for _ in range(10):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
clear_output(wait = True)

/var/folders/6f/c2b7vdpx247cstzj573kd1k40000gn/T/ipykernel_84694/3314136448.py:1: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


Request: 1; Frequency: 0.9948208550949447 requests/s
Request: 2; Frequency: 0.4991454509028029 requests/s
Request: 3; Frequency: 0.4991799158080749 requests/s
Request: 4; Frequency: 0.4439102173576265 requests/s
Request: 5; Frequency: 0.41621208169946705 requests/s
Request: 6; Frequency: 0.3995464006866543 requests/s
Request: 7; Frequency: 0.4113082720485279 requests/s
Request: 8; Frequency: 0.42059833849625333 requests/s
Request: 9; Frequency: 0.44947199976681446 requests/s
Request: 10; Frequency: 0.4342480058678245 requests/s


In [4]:
from warnings import warn
warn("Warning Simulation")

/var/folders/6f/c2b7vdpx247cstzj573kd1k40000gn/T/ipykernel_84694/1729884990.py:2: UserWarning: Warning Simulation
  warn("Warning Simulation")


In [5]:
from requests import get

In [6]:
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [7]:
from bs4 import BeautifulSoup

In [8]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
years_url = [str(i) for i in range(2000,2022)]
pages = [str(i) for i in range(1,10)]
# Preparing the monitoring of the loop

genres=[]
requests = 0
runtimes=[]

# For every year in the interval 2000-2022
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('https://www.imdb.com/search/title?release_date=' + year_url +
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 200:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
                
            if container.p.find('span', class_ = 'genre') is not None:
                genre = container.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
                genres.append(genre)
            else:
                genres.append("")
          

Request:198; Frequency: 0.07220846478590529 requests/s


In [9]:
import pandas as pd

In [10]:
len(years)

8667

In [11]:
genres= genres[:8667]

In [12]:
len(genres)

8667

In [18]:
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'genres': genres,
})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8667 entries, 0 to 8666
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      8667 non-null   object 
 1   year       8667 non-null   object 
 2   imdb       8667 non-null   float64
 3   metascore  8667 non-null   int64  
 4   votes      8667 non-null   int64  
 5   genres     8667 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 406.4+ KB
None


,movie,year,imdb,metascore,votes,genres
0,Gladiator,(2000),8.5,67,1498527,"[Action, Adventure, Drama]"
1,Memento,(2000),8.4,81,1237640,"[Mystery, Thriller]"
2,Snatch,(2000),8.2,55,856901,"[Comedy, Crime]"
3,Requiem for a Dream,(2000),8.3,68,842800,[Drama]
4,American Psycho,(2000),7.6,64,621197,"[Crime, Drama, Horror]"
5,X-Men,(2000),7.3,64,617799,"[Action, Adventure, Sci-Fi]"
6,Cast Away,(2000),7.8,73,594097,"[Adventure, Drama, Romance]"
7,Unbreakable,(2000),7.3,62,419963,"[Drama, Mystery, Sci-Fi]"
8,Mission: Impossible II,(2000),6.1,59,344674,"[Action, Adventure, Thriller]"
9,Meet the Parents,(2000),7.0,73,337484,"[Comedy, Romance]"


In [19]:
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes','genres']]
movie_ratings.head()

,movie,year,imdb,metascore,votes,genres
0,Gladiator,(2000),8.5,67,1498527,"[Action, Adventure, Drama]"
1,Memento,(2000),8.4,81,1237640,"[Mystery, Thriller]"
2,Snatch,(2000),8.2,55,856901,"[Comedy, Crime]"
3,Requiem for a Dream,(2000),8.3,68,842800,[Drama]
4,American Psycho,(2000),7.6,64,621197,"[Crime, Drama, Horror]"


In [20]:
movie_ratings['genres'].astype(str)

0         ['Action', ' Adventure', ' Drama']
1                   ['Mystery', ' Thriller']
2                       ['Comedy', ' Crime']
3                                  ['Drama']
4             ['Crime', ' Drama', ' Horror']
                        ...                 
8662    ['Action', ' Adventure', ' Fantasy']
8663      ['Action', ' Biography', ' Drama']
8664            ['Comedy', ' Drama', ' War']
8665       ['Biography', ' Crime', ' Drama']
8666       ['Action', ' Crime', ' Thriller']
Name: genres, Length: 8667, dtype: object

In [46]:
for a in range(len(movie_ratings['genres'])):
    movie_ratings['genres'][a]=','.join(movie_ratings['genres'][a])

/var/folders/6f/c2b7vdpx247cstzj573kd1k40000gn/T/ipykernel_84694/1452348974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_ratings['genres'][a]=','.join(movie_ratings['genres'][a])


In [47]:
movie_ratings['genres']

0         Action, Adventure, Drama
1                Mystery, Thriller
2                    Comedy, Crime
3                            Drama
4             Crime, Drama, Horror
                   ...            
8662    Action, Adventure, Fantasy
8663      Action, Biography, Drama
8664            Comedy, Drama, War
8665       Biography, Crime, Drama
8666       Action, Crime, Thriller
Name: genres, Length: 8667, dtype: object

In [23]:
movie_ratings['year'].unique()

array(['(2000)', '(I) (2000)', '(2001)', '(2002)', '(2003)', '(2004)',
       '(I) (2004)', '(2005)', '(I) (2005)', '(2006)', '(I) (2006)',
       '(2007)', '(I) (2007)', '(2008)', '(I) (2008)', '(2009)',
       '(I) (2009)', '(2010)', '(I) (2010)', '(2011)', '(I) (2011)',
       '(2012)', '(I) (2012)', '(2013)', '(I) (2013)', '(2014)',
       '(I) (2014)', '(II) (2014)', '(2015)', '(I) (2015)', '(II) (2015)',
       '(2016)', '(II) (2016)', '(IX) (2016)', '(I) (2016)', '(2017)',
       '(I) (2017)', '(2018)', '(I) (2018)', '(III) (2018)', '(I) (2019)',
       '(2019)', '(II) (2019)', '(2020)', '(I) (2020)', '(II) (2020)',
       '(2021)', '(I) (2021)'], dtype=object)

In [24]:
movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(int)

/var/folders/6f/c2b7vdpx247cstzj573kd1k40000gn/T/ipykernel_84694/4256737050.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(int)


In [25]:
movie_ratings.describe().loc[['min', 'max'], ['imdb', 'metascore']]

,imdb,metascore
min,4.1,24.0
max,9.0,100.0


In [26]:
movie_ratings['n_imdb'] = movie_ratings['imdb'] * 10
movie_ratings.head(3)

,movie,year,imdb,metascore,votes,genres,n_imdb
0,Gladiator,2000,8.5,67,1498527,"[Action, Adventure, Drama]",85.0
1,Memento,2000,8.4,81,1237640,"[Mystery, Thriller]",84.0
2,Snatch,2000,8.2,55,856901,"[Comedy, Crime]",82.0


In [27]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [28]:
import mysql.connector as msql
from mysql.connector import Error
try:
    conn = msql.connect(host='localhost', user='root',  
                        password='Anita')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE imdb;")
        cursor.execute("select database();")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'imdb'; database exists


In [48]:
conn = msql.connect(host='localhost', database='estes', user='root', password='Anita')
if conn.is_connected():
    print("Connection is ..")
    cursor = conn.cursor()

    cursor.execute("use imdb;")

    record = cursor.fetchone()
    #print("You're connected to database: ", record)
    cursor.execute('DROP TABLE IF EXISTS movie_reviews;')
    print('Creating table....')
# in the below line please pass the create table statement which you want #to create

    sql1= """CREATE TABLE movie_reviews(
    movie varchar(255),
    year varchar(255),
    imdb float,
    metascore int,
    votes int,
    genre varchar(255),
    n_imdb float
    );"""
    cursor.execute(sql1)
    


Connection is ..
Creating table....


In [49]:
for i,row in movie_ratings.iterrows():
            #here %S means string values 
            sql = "INSERT INTO imdb.movie_reviews VALUES (%s,%s,%s,%s,%s,%s,%s);"
            #print(i,row)
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()


Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

...

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

In [50]:
# APPLY Neural nets on it
# publish to github


NameError: name 'movie_ratings' is not defined